In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/stock_price_4_fun/notebook


In [2]:
import requests
import re
import time
import random
import urllib3

urllib3.disable_warnings()

In [3]:
from src.summarize_text import *

In [4]:
speech_to_text = SpeechSummaryProcessor(audio_path='data/audio_ogg.ogg')
text = speech_to_text.generate_speech_to_text()

Device cuda:0


In [5]:
text

"1: I'll pick up the things at 8 o'clock later\n\n2: At 9:00 a.m., I go to bed to write my essay and go to bed at 10:00 a.m\n"

In [6]:
symbol = 'SSI'

In [7]:
news_scraper = NewsScraper()
news_list = news_scraper.search_stock_news(symbol=symbol, date_format='year')
news = news_scraper.take_text_from_link(news_url=news_list[0])

In [8]:
news

'Tâm điểm đáng chú ý gần đây là việc Ngân hàng Nhà nước quay lại phát hành tín phiếu trên kênh thị trường mở, với tổng khối lượng là 70.000 tỷ đồng trong 5 phiên liên tiếp. Khối lượng phát hành ban đầu ở mức 10.000 tỷ đồng cho ba phiên đầu tiên, sau đó tăng lên 20.000 tỷ đồng trong hai phiên gần nhất.\nLo ngại khả năng đảo chiều chính sách tiền tệ theo hướng thắt chặt, theo giới chuyên gia, là một phần nguyên nhân khiến chứng khoán đỏ lửa trong một tháng gần đây. Tuy nhiên, trong báo cáo mới phát hành, Bộ phận phân tích Công ty chứng khoán SSI (SSI Research) cho rằng đây là hoạt động thường thấy nhằm điều chỉnh trạng thái thanh khoản ngắn hạn của hệ thống.\n"Động thái này không đồng nghĩa với việc Ngân hàng Nhà nước sẽ đảo chiều chính sách tiền tệ", nhóm phân tích nhận xét.\nTrên thực tế, việc phát hành tín phiếu còn có thể được coi là tích cực, thay vì Ngân hàng Nhà nước lựa chọn phương án bán ngoại tệ từ dự trữ ngoại hối. Thông qua nghiệp vụ này, cơ quan quản lý có thể có những đánh 

In [9]:
new_summarizer = NewsSummarizer()
sum_text = new_summarizer.summary_news(news= news)

In [16]:
print(sum_text)

Việc NHNN quay trở lại phát hành tín phiếu trên kênh thị trường mở có tổng khối lượng 70.000 tỷ đồng. 
Khối lượng phát hành ban đầu ở mức 10.000 tỷ đồng trong ba phiên đầu tiên, sau đó tăng lên 20.000 tỷ đồng trong hai phiên cuối cùng. 
Lo ngại về khả năng đảo ngược chính sách tiền tệ theo hướng thắt chặt là một phần nguyên nhân khiến chứng khoán giảm giá trong những tháng gần đây.

Ngân hàng Nhà nước chọn phát hành tín phiếu là phương án bắt đầu từ năm 2023. 
Chênh lệch cung cầu trên thị trường ngân hàng ngày càng chênh lệch lớn. 
SSI Research cho rằng con số 70.000 tỷ đồng vẫn còn thấp.



In [12]:
watch_list=['GEX', 'VIX', 'HHV', 'BSR', 'SHS', 'PDR', 'CTR', 'HAH', 'VNINDEX', 'VND']

In [13]:
news_db = StockNewsDatabase()
# news_db.update_stock_news(watch_list)

In [20]:
news_db.extract_text_for_stock(stock_symbol='HAH')

('Cổ phiếu GMD của Gemadept là một trong những cổ phiếu tăng mạnh nhất sàn HoSE khi đóng cửa tăng toàn bộ lên 73.000 đồng. \nCác cổ phiếu cảng biển, logistics khác cũng giao dịch sôi động như HAH tăng gần 4%. \nChứng khoán cũng mở cửa phiên đầu tuần trong trạng thái tích cực.\n\nMSN là cổ phiếu duy nhất tăng trên 5%, MWG tăng thêm 0,7%, FRT và DGW đều đóng cửa trong sắc xanh. \nỞ nhóm bất động sản, màu xanh cũng chiếm ưu thế.\n',
 'https://vnexpress.net/co-phieu-van-tai-bien-noi-song-4693192.html')

In [15]:
news_db.get_all_stocks()

['VND', 'MSB', 'STB', 'HAH', 'CEO', 'BSR', 'HPG', 'PVS']

In [17]:
# news_db.update_stock_news(watch_list)

Your max_length is set to 230, but your input_length is only 75. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


Summary data saved to data/summary_stock_news.json
